In [ ]:
import sys
sys.path.append("../../digitech_classify")

import pandas as pd
from digitech_classify.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR





In [ ]:
df1 = pd.read_csv(PROCESSED_DATA_DIR / "company_tagged_all-MiniLM-L6-v2.csv")
df2 = pd.read_csv(PROCESSED_DATA_DIR / "companies_tagged_en_core_web_lg.csv")        
df3 = pd.read_csv(PROCESSED_DATA_DIR / "tech_keyword_match_lemaspacey.csv")

In [ ]:
df3 = df3[df3['data_source'] != 'Net0']

print(df1.shape, df2.shape, df3.shape)
print(df1.columns, df2.columns, df3.columns)

In [ ]:

df1 = df1.rename(columns={'keyword_miniLM': 'keyword'})
df2 = df2.rename(columns={'keyword_web_lg': 'keyword'})
df3 = df3.rename(columns={'matched_keyword': 'keyword'})  


df1_sub = df1[['org_ID', 'organisation_name', 'keyword']]
df2_sub = df2[['org_ID', 'organisation_name', 'keyword']]
df3_sub = df3[['org_ID', 'organisation_name', 'keyword']]



final_df = pd.concat([df1_sub, df2_sub, df3_sub], ignore_index=True)




In [ ]:
print(final_df.shape)

In [ ]:

kw_df = pd.read_csv(INTERIM_DATA_DIR / "keywords_lemma.csv")
kw_df.columns


In [ ]:
final_df['keyword'] = final_df['keyword'].astype(str).str.strip().str.lower()
kw_df['keywords_lemma'] = kw_df['keywords_lemma'].astype(str).str.strip().str.lower()

final_df = final_df.merge(
    kw_df[['Sector', 'Keyword', 'keywords_lemma']],
    left_on='keyword',
    right_on='keywords_lemma',
    how='left'
)


In [ ]:

final_df = final_df.drop(columns=['Keyword'])
final_df = final_df[['org_ID', 'organisation_name', 'keyword', 'keywords_lemma', 'Sector']]

In [ ]:
raw_df = pd.read_csv(INTERIM_DATA_DIR / "cleaned_companies_text.csv")
print(raw_df.columns)

In [ ]:


final_df = final_df.merge(
    raw_df[['org_ID', 'search_text']],
    on='org_ID',
    how='left'
)

In [ ]:
print(final_df['Sector'].unique())

In [ ]:
sectors = [
    'cybersecurity', 'cloud-edge-iot', 'artificial intelligence',
    'data analytics', 'next generation internet and extended reality',
    'blockchain', 'advanced digital communications and connectivity',
    'high performance computing', 'photonics', 'robotics',
    'quantum technologies'
]

samples = []
for sector in sectors:
    sector_df = final_df[final_df['Sector'] == sector].sample(n=100, random_state=42, replace=False)
    samples.append(sector_df)

sampled_df = pd.concat(samples, ignore_index=True)


sampled_df.to_excel(PROCESSED_DATA_DIR / "sampled_100_per_sector.xlsx", index=False)

In [ ]:
final_df.to_csv(PROCESSED_DATA_DIR / "final_dataset.csv", index=False)